# LaTeX Automation for NYCHA Waste Individual Action Plans

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from PIL import Image
import docx2txt
import re

In [2]:
#Set Global Vars and Options
os.chdir('/Users/kyleslugg/Documents/NYCHA/Production')
cons_tds = '073'
pd.set_option('display.max_columns', None)

## Parse and Process Text Blocks

TO COME:
- Waste Services and Assets
- Waste Distribution (top text -- bottom accounted for below)
- What Is an IAP?

In [ ]:
#Character Substitutions for LaTeX -- set and define "clean" method
def clean_text(text):
    substitutions = {'“':"``",
                '”': "''",
                '’':"'",
                ' ':' ',
                '–':'--',
                ' ':' ',
                '\xa0':' '}
    
    for key, value in substitutions.items():
        text = text.replace(key, value)
        
    return text

#### Overview Text

In [ ]:
def make_overview_text(cons_tds):
    header = re.compile(r'((\w*\s)*(Overview)):?')
    
    overview_text = docx2txt.process(f'TEXT/{cons_tds}_Overview.docx')
    try:
        overview_text = overview_text.replace(header.findall(overview_text)[0]+'\n','')
    except:
        overview_text = overview_text.replace(header.findall(overview_text)[0][0]+'\n','')
    overview_text = clean_text(overview_text)
    
    with open(f'TEXT/overview_text/{cons_tds}_overview.tex', 'w') as file_handle:
        file_handle.write(overview_text)
    
    pass

In [ ]:
make_overview_text(cons_tds)

#### Analysis Text

In [ ]:
def make_analysis_text(cons_tds):
    analysis_text = docx2txt.process(f'TEXT/{cons_tds}_Analysis.docx')

    header = re.compile(r'([\w\s]*:)')

    section_headings = {'Inspection and Collection Requirement':['Inspection and Collection Requirement',
                                                                'Inspection and Collection Requirements',
                                                                'Collection and Inspection Requirement',
                                                                'Collection and Inspection Requirements'],
                        
                        'Removal or Storage Requirement':['Removal or Storage Requirement',
                                                          'Removal or Storage Requirements',
                                                         'Removal and Storage Requirement',
                                                         'Removal and Storage Requirements',
                                                         'Storage or Removal Requirement',
                                                         'Storage and Removal Requirement',
                                                         'Storage and Removal Requirements']}
    
    for heading, variants in section_headings.items():
        for variant in variants:
            analysis_text = analysis_text.replace(variant, r'\emph{%s}' % heading)

    analysis_text = analysis_text.replace(header.findall(analysis_text)[0]+'\n','')

    latex_block = clean_text(analysis_text)

    with open(f'TEXT/analysis_text/{cons_tds}_analysis.tex', 'w') as file_handle:
        file_handle.write(latex_block)
        
    pass

In [ ]:
make_analysis_text(cons_tds)

## Select and Prepare Maps

In [ ]:
# Set asset map path
asset_map_path = f"MAPS/asset_maps/{cons_tds}_asset_map.png"

In [ ]:
# Split context map into two pages
def process_context_map(cons_tds):
    image = Image.open(f'REPORT_TEMPLATE/{cons_tds}_context_map.png')
    width, height = image.size

    bb1 = (0,0,width/2,height)
    bb2 = (width/2, 0, width, height)

    img_1 = image.crop(bb1)
    img_2 = image.crop(bb2)

    img_1.save(f'MAPS/context_maps/{cons_tds}_context_1.png', format="PNG")
    img_2.save(f'MAPS/context_maps/{cons_tds}_context_2.png', format="PNG")
    
    pass

## Produce Tables

TO COME:
- Waste Services and Assets
- Assets
- Capital Improvements

In [3]:
#Load Data
overview_data = pd.read_csv('DATA/overview_table_data.csv')
overview_data['CONS_TDS'] = overview_data['CONS_TDS'].apply(lambda x: str(x).zfill(3))
overview_data['TDS'] = overview_data['TDS'].apply(lambda x: str(x).zfill(3))

#### Make Overview Table

In [ ]:
def make_overview_table(cons_tds, overview_data=overview_data):
    
    cons_data = overview_data.loc[overview_data['CONS_TDS']== cons_tds]
    
    overview_table = ''

    overview_frame = r'''
    \begin{tabular}{l|c|c|c|c|}
    \cline{2-5}
                                                                           & \cellcolor{ccteal}{\color[HTML]{FFFFFF} TDS \#} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Total Households} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Official Population} & \cellcolor{ccteal}{\color[HTML]{FFFFFF} Average Family Size} \\ \hline

    '''

    development_template = r'''\multicolumn{1}{|l|}{\cellcolor{ccteallight}%s}        & %s                                                   & %s                                                           & %s                                                                & %s                                                                \\ \hline'''


    overview_table += overview_frame

    for row in cons_data.itertuples():
        dev_name = row.DEV_NAME.title()
        dev_tds = row.TDS
        total_hhs = row.TOTAL_HH
        official_population = row.TOTAL_POP
        avg_family_size = row.AVG_FAMILY_SIZE

        overview_table += development_template % (dev_name, dev_tds, total_hhs, official_population, avg_family_size)

    overview_table += r'''
    \end{tabular}
    '''
    
    with open(f'TABLES/overview_table/{cons_tds}_overview_table.tex', 'w') as file_handle:
        file_handle.write(overview_table)
    

#### Waste Calculator

In [5]:
def add_waste_cols(overview_data=overview_data):
    conversion_factors = {'units_to_tons_day': 0.0025,
                         'cy_per_ton': {'trash': 21.05,
                                       'MGP': 18.02,
                                       'cardboard': 26.67,
                                       'paper': 6.19,
                                       'organics': 4.32,
                                       'ewaste': 5.65,
                                       'textiles': 13.33},
                         'gallons_per_cy': 201.974,
                         'gallons_per_64gal': 64,
                         'gallons_per_40lb_bag': 44}

    waste_percentages = {'trash': .26,
                        'MGP': .19,
                        'cardboard': .07,
                        'paper': .07,
                        'organics':.32,
                        'ewaste': .01,
                        'textiles': .08}

    capture_rates = {'trash_primary': .75,
                    'trash_secondary': .25,
                    'mgp': .05,
                    'cardboard': .25,
                    'paper': .10}

    overview_data['WASTE_TONS_DAY'] = overview_data['CURRENT_APTS'].apply(lambda x: x * conversion_factors['units_to_tons_day'])

    for key, value in waste_percentages.items():
        overview_data[f'{key.upper()}_CY'] = overview_data['WASTE_TONS_DAY'].apply(lambda x: x * value * conversion_factors['cy_per_ton'][key])

    overview_data['TRASH_CHUTE_CY'] = overview_data['TRASH_CY']*capture_rates['trash_primary']
    overview_data['TRASH_CHUTE_SAUSAGE'] = ((overview_data['TRASH_CHUTE_CY'])/conversion_factors['cy_per_ton']['trash'])*(2000/40)
    overview_data['TRASH_DROP_CY'] = overview_data['TRASH_CY']*capture_rates['trash_secondary']
    overview_data['TRASH_DROP_BINS'] = overview_data['TRASH_DROP_CY']*conversion_factors['gallons_per_cy']/64
    overview_data['CAPTURED_MGP_CY'] = overview_data['MGP_CY']*capture_rates['mgp']
    overview_data['CAPTURED_CARDBOARD_CY'] = overview_data['CARDBOARD_CY']*capture_rates['cardboard']
    overview_data['CAPTURED_PAPER_CY'] = overview_data['PAPER_CY']*capture_rates['paper']

    return overview_data

In [30]:
def make_waste_distribution_table(cons_tds, overview_data=overview_data):
    cons_data = overview_data[overview_data['CONS_TDS'] == cons_tds]
    num_devs = cons_data.shape[0]
    
    if num_devs > 1:
        num_cols = num_devs+1
    else:
        num_cols = num_devs

    dev_col_format = r'V{1.25in}|'

    opening = r'''
    \begin{tabular*}{\textwidth}{V{1.5in}|@{\extracolsep{\fill}}%s}
    \cline{2-%s}
    ''' % (dev_col_format*num_cols, (num_cols+1))

    top_row = r'''
                                                                   '''+(r"& \multicolumn{1}{l|}{\cellcolor{ccorange}%s}"*(num_cols))+r"\tnhl"+'\n'

    standard_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s}                 "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    captured_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Captured / Day (CY)\tnote{4}}                        "+(r"& %s                                    ")*num_cols+r"\tnhl"+'\n'

    chute_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Trash Chutes\tnote{2}}                 "+(r"& %s or (%s) 40 lbs. sausage bags      "*num_cols)+r"\tnhl"+'\n'

    dropsite_row = r"\multicolumn{1}{|Y{1.5in}|}{\cellcolor{ccorangelight}Drop Sites\tnote{3}}                 "+(r"& %s or (%s) 64-gallon bins      "*num_cols)+r"\tnhl"+'\n'

    OET_row = r"\multicolumn{1}{|V{1.5in}|}{\cellcolor{ccorangelight}%s / Day (CY)\tnote{5}}              "+(r"& %s                                    "*num_cols)+r"\tnhl"+'\n'
    
    def make_waste_distribution_table_block(cons_data, num_cols):
        if num_cols != 1:
            cons_data.loc['Total']= cons_data.sum(numeric_only=True, axis=0)
            cons_data.loc['Total','DEV_NAME'] = 'Total'

        def make_trash_text(row, text_var, cy_col, other_col):
            text_var.append(round(row[cy_col],2))
            text_var.append(round(row[other_col], 2))
            pass

        latex_block = opening
        latex_block += top_row % tuple(cons_data['DEV_NAME'].apply(lambda x: str(x).title()).tolist())
        latex_block += standard_row % tuple([r"Waste Generated / Day (Tons)\tnote{1}"]+[round(item, 2) for item in cons_data['WASTE_TONS_DAY'].tolist()])
        latex_block += standard_row % tuple([r"Trash / Day (CY)"]+cons_data['TRASH_CY'].apply(lambda x: str(round(x,2))).tolist())

        trash_chute_text = []
        dropsite_text = []

        cons_data.apply(lambda row: make_trash_text(row, trash_chute_text, 'TRASH_CHUTE_CY', 'TRASH_CHUTE_SAUSAGE'), axis=1)
        cons_data.apply(lambda row: make_trash_text(row, dropsite_text, 'TRASH_DROP_CY', 'TRASH_DROP_BINS'), axis=1)

        latex_block += chute_row % tuple(trash_chute_text)
        latex_block += dropsite_row % tuple(dropsite_text)

        latex_block += standard_row % tuple([r"Metal, Glass, Plastic / Day (CY)"]+cons_data['MGP_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += captured_row % tuple(cons_data['CAPTURED_MGP_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += standard_row % tuple([r"Cardboard / Day (CY)"]+cons_data['CARDBOARD_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += captured_row % tuple(cons_data['CAPTURED_CARDBOARD_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += standard_row % tuple([r"Paper / Day (CY)"]+cons_data['PAPER_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += captured_row % tuple(cons_data['CAPTURED_PAPER_CY'].apply(lambda x: str(round(x,2))).tolist())

        latex_block += OET_row % tuple(['Organics']+cons_data['ORGANICS_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += OET_row % tuple(['E-Waste']+cons_data['EWASTE_CY'].apply(lambda x: str(round(x,2))).tolist())
        latex_block += OET_row % tuple(['Textiles']+cons_data['TEXTILES_CY'].apply(lambda x: str(round(x,2))).tolist())

        latex_block += r"\end{tabular*}"

        return latex_block
    
    
    latex_block = make_waste_distribution_table_block(cons_data, num_cols)
    
    with open(f'TABLES/waste_distribution_table/{cons_tds}_wd_table.tex', 'w') as file_handle:
        file_handle.write(latex_block)
    
    text_block = r''''''
    
    text_line_multi = r'''\bf{%s}: This development has %s apartment units and %s stairhalls.\\
    
    
    '''
    
    text_line_singular = r'''\bf{%s}: This development has %s apartment units and one stairhall.\\
    
    
    '''
    
    def make_text_line(row):
        if int(row['STAIRHALLS']) == 1:
            text_block += text_line_singluar % (row['DEV_NAME'].title(), row['CURRENT_APTS'])
        else:
            text_block += text_line % (row['DEV_NAME'].title(), row['CURRENT_APTS'], int(row['STAIRHALLS']))
        
        pass
    
    cons_data.apply(lambda row: make_text_line(row), axis=1)
    
    with open(f'TEXT/waste_distribution_bottom/{cons_tds}_wd_bottom.tex', 'w') as file_handle:
        file_handle.write(text_block)
    
    

#### Make Capital Improvements Table

In [ ]:
fwd = pd.read_csv('DATA/capital_fwd.csv')
ehd = pd.read_csv('DATA/capital_ehd.csv')
int_compactor = pd.read_csv('DATA/capital_intcom.csv')
wasteyard = pd.read_csv('DATA/capital_wasteyard.csv')

#### Make Staff Table

In [ ]:
cons_ids = pd.read_csv('DATA/CONS_NAME_TDS.csv')
cons_ids['CONS_TDS'] = cons_ids['CONS_TDS'].apply(lambda x: str(x).zfill(3))

dev_staff = pd.read_csv('DATA/staff_for_table.csv')
dev_staff.dropna(inplace=True)
dev_staff['Consolidation'] = dev_staff['Consolidation'].apply(lambda x: str(x).upper())
#Note: Staff list missing for Armstrong, Ft. Washington, and Williams Plaza, as well as scatter-site third-party-managed consolidations
dev_staff = dev_staff.merge(cons_ids, left_on='Consolidation', right_on='CONS_NAME', how='inner', indicator=False)

In [ ]:
table_frame = pd.read_csv('DATA/Table_Keys.csv')

In [ ]:
def make_staff_table(cons_tds):
    cons_id = cons_ids['CONS_NAME'].loc[cons_ids['CONS_TDS'] == cons_tds].iloc[0]
    
    #Fetching staff data for consolidation
    cons_data = dev_staff.loc[dev_staff['Consolidation'] == cons_id]
    
    #Setting up table and transposing data
    cons_table_frame = table_frame
    cons_table_frame['Budgeted'] = cons_table_frame['BUDG_KEY'].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame['Formula'] = cons_table_frame['FORMULA_KEY'].iloc[:-1].apply(lambda key: cons_data[key].iloc[0])
    cons_table_frame.loc[cons_table_frame['CHART_LINE']== 'Caretakers J', 'Variance'] = (cons_table_frame.iloc[-1]['Budgeted']+
                                             cons_table_frame.iloc[-2]['Budgeted']-
                                             cons_table_frame.iloc[-2]['Formula'])
    
    #Simplifying table
    cons_table = cons_table_frame[['CHART_LINE', 'Budgeted', 'Formula', 'Variance']]
    
    #Defining LaTeX table format
    
    def make_staff_table_block(staff_data):
    
        table_template = r'''
        \begin{tabular}{l|c|c|c|}
        \cline{2-4}
                                                                                     & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Budgeted} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Formula Allocation} & \cellcolor{ccfuschia}{\color[HTML]{FFFFFF} Variance} \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Employees}                      & %s                                                      & %s                                                                & %s                                                        \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Property Manager}               & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Asst. Property Manager}         & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Secretaries}                    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Housing Assistants}             & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Superintendent}                 & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Assistant Superintendent}       & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Caretakers (SOC)} & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Supervisor of Grounds (SOG)}    & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Maintenance Workers}            & %s                                                      & %s                                                                & %s                                                       \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers X}                   & %s                                                      & %s                                                                & %s                                                      \\ \hline
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers J\tnote{1}}                   & %s                                                      &                                                                 &                                                         \\ \cline{1-2}
        \multicolumn{1}{|l|}{\cellcolor{ccfuschialight}Caretakers G}                   & %s                                                      & \multirow{-2}{*}{%s}                                     & \multirow{-2}{*}{%s}                           \\ \hline
        \end{tabular}
        
        '''

        values = []

        def extract_data_through_ctx(row):
            [values.append(item) for item in [str(int(row['Budgeted'])), 
                                              str(int(row['Formula'])), 
                                              str(int(row['Variance']))]]
            pass

        #Processing through Caretaker X
        staff_data.iloc[0:-2].apply(lambda row: extract_data_through_ctx(row), axis=1)

        #Processing Caretaker J and G
        values.append(str(int(staff_data.iloc[-2, 1])))
        values.append(str(int(staff_data.iloc[-1, 1])))
        values.append(str(int(staff_data.iloc[-2, 2])))
        values.append(str(int(staff_data.iloc[-2, 3])))

        return table_template % tuple(values)
    
    #Make and export LaTeX code
    with open(f'TABLES/staff_table/{cons_tds}_staff_table.tex', 'w') as file_handle:
        file_handle.write(make_staff_table_block(cons_table))